In [1]:
# ==============================
# Lab 09 – Stacking + Pipeline + LIME (Corrected)
# ==============================

import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import lime
import lime.lime_tabular
from functools import reduce

# Suppress warnings for clean output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# ------------------------------
# Function: Load Data
# ------------------------------
def load_data():
    """
    Loads, merges, and preprocesses the project-specific datasets.
    This version reads gage height, precipitation, reservoir storage, and stream flow data,
    merges them by datetime, and creates a binary classification target for stream flow.
    """

    # Helper function to parse the specific USGS CSV format
    def parse_usgs_file(filepath, value_name):
        try:
            # The files have a format where each line is a single quoted string containing tabs.
            # 1. Read the file as a single column, skipping commented header lines.
            df = pd.read_csv(filepath, header=None, comment='#')

            if df.empty:
                return pd.DataFrame({'datetime': [], value_name: []})

            # 2. Strip quotes and split the single column into multiple columns by tab.
            data = df[0].str.strip('"').str.split('\t', expand=True)

            # 3. Assign column names based on the file format description.
            column_names = ['agency_cd', 'site_no', 'datetime_str', 'tz_cd', 'value', 'prov_qual_cd']
            data.columns = column_names[:data.shape[1]]

            # 4. Keep relevant columns, convert types, and rename the value column.
            df_clean = data[['datetime_str', 'value']].copy()
            df_clean.rename(columns={'datetime_str': 'datetime', 'value': value_name}, inplace=True)

            df_clean['datetime'] = pd.to_datetime(df_clean['datetime'])
            df_clean[value_name] = pd.to_numeric(df_clean[value_name], errors='coerce')

            return df_clean
        except FileNotFoundError:
            print(f"Error: The file '{filepath}' was not found.")
            return None

    # File paths for the datasets
    files = {
        'gage_height': 'gage_height.csv',
        'precipitation': 'precipitation.csv',
        'reservoir_storage': 'reservoir_storage.csv',
        'stream_flow': 'stream_flow.csv'
    }

    # Load and parse each file
    df_gage = parse_usgs_file(files['gage_height'], 'gage_height')
    df_precip = parse_usgs_file(files['precipitation'], 'precipitation')
    df_res = parse_usgs_file(files['reservoir_storage'], 'reservoir_storage')
    df_flow = parse_usgs_file(files['stream_flow'], 'stream_flow')

    # Filter out any dataframes that failed to load
    data_frames = [df for df in [df_gage, df_precip, df_res, df_flow] if df is not None]

    if len(data_frames) < 4:
        print("Could not load all necessary data files. Exiting.")
        exit()

    # Merge the dataframes on the 'datetime' column
    df_merged = reduce(lambda left, right: pd.merge(left, right, on='datetime', how='inner'), data_frames)

    # Drop any rows with missing values
    df_merged.dropna(inplace=True)

    # Define features (X) and the target variable (y)
    feature_names = ['gage_height', 'precipitation', 'reservoir_storage']
    X = df_merged[feature_names]

    # Create a binary classification target from 'stream_flow'
    # "High flow" is anything above the median stream flow.
    flow_threshold = df_merged['stream_flow'].median()
    y = (df_merged['stream_flow'] > flow_threshold).astype(int)

    # Define class names for LIME explainer
    class_names = ['Low Flow', 'High Flow']

    return X, y, feature_names, class_names


# ------------------------------
# Function: Build Stacking Model
# ------------------------------
def build_stacking_classifier():
    """Builds a Stacking Classifier with base models and a meta-model."""
    base_models = [
        ("rf", RandomForestClassifier(n_estimators=50, random_state=42)),
        ("gb", GradientBoostingClassifier(n_estimators=50, random_state=42))
    ]
    meta_model = LogisticRegression()

    return StackingClassifier(
        estimators=base_models,
        final_estimator=meta_model,
        passthrough=True
    )


# ------------------------------
# Function: Build Pipeline
# ------------------------------
def build_pipeline(model):
    """Builds a scikit-learn pipeline with a scaler and a model."""
    return Pipeline(steps=[
        ("scaler", StandardScaler()),
        ("model", model)
    ])


# ------------------------------
# Function: Train & Evaluate
# ------------------------------
def train_and_evaluate(pipeline, X_train, X_test, y_train, y_test):
    """Fits the pipeline and returns accuracy, report, and the fitted pipeline."""
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_test)
    acc = accuracy_score(y_test, preds)
    report = classification_report(y_test, preds, target_names=['Low Flow', 'High Flow'])
    return acc, report, pipeline


# ------------------------------
# Function: Explain with LIME
# ------------------------------
def explain_with_lime(pipeline, X_train, X_test, feature_names, class_names):
    """
    Generates a LIME explanation for a single test sample.
    This function now returns the explanation details instead of printing them.
    """
    explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values,
        feature_names=feature_names,
        class_names=class_names,
        mode="classification"
    )

    sample_index = 0
    sample_to_explain = X_test.iloc[sample_index]

    explanation = explainer.explain_instance(
        data_row=sample_to_explain.values,
        predict_fn=pipeline.predict_proba
    )

    explanation_details = {
        "sample_index": sample_index,
        "true_features": sample_to_explain.to_dict(),
        "predicted_probabilities": pipeline.predict_proba([sample_to_explain])[0],
        "explanation_list": explanation.as_list()
    }

    return explanation_details


# ------------------------------
# Main Program
# ------------------------------
if __name__ == "__main__":
    X, y, feature_names, class_names = load_data()

    if X.empty:
        print("Data loading resulted in an empty dataset. Cannot proceed.")
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        stacking_model = build_stacking_classifier()
        pipeline = build_pipeline(stacking_model)

        acc, report, fitted_pipeline = train_and_evaluate(pipeline, X_train, X_test, y_train, y_test)

        print("====================================")
        print("      MODEL EVALUATION RESULTS      ")
        print("====================================")
        print(f"✅ Accuracy: {acc:.4f}")
        print("\n📋 Classification Report:\n", report)

        lime_explanation = explain_with_lime(fitted_pipeline, X_train, X_test, feature_names, class_names)

        print("\n====================================")
        print("         LIME EXPLANATION         ")
        print("====================================")
        print(f"🔎 LIME Explanation for Test Sample #{lime_explanation['sample_index']}")
        print("\n--- True Feature Values ---")
        for feature, value in lime_explanation['true_features'].items():
            print(f"  {feature}: {value}")

        print("\n--- Predicted Probabilities ---")
        for i, class_name in enumerate(class_names):
            prob = lime_explanation['predicted_probabilities'][i]
            print(f"  P(class={class_name}) = {prob:.3f}")

        print("\n--- Feature Contributions (Top) ---")
        for feature, weight in lime_explanation['explanation_list']:
            print(f"  {feature:<25} --> Contribution: {weight:.3f}")
        print("====================================")



      MODEL EVALUATION RESULTS      
✅ Accuracy: 0.9569

📋 Classification Report:
               precision    recall  f1-score   support

    Low Flow       1.00      0.92      0.96       909
   High Flow       0.92      1.00      0.96       901

    accuracy                           0.96      1810
   macro avg       0.96      0.96      0.96      1810
weighted avg       0.96      0.96      0.96      1810


         LIME EXPLANATION         
🔎 LIME Explanation for Test Sample #0

--- True Feature Values ---
  gage_height: 20.23
  precipitation: 0.0
  reservoir_storage: 34.38

--- Predicted Probabilities ---
  P(class=Low Flow) = 0.064
  P(class=High Flow) = 0.936

--- Feature Contributions (Top) ---
  34.37 < reservoir_storage <= 34.87 --> Contribution: 0.426
  precipitation <= 0.00     --> Contribution: -0.130
  20.19 < gage_height <= 20.23 --> Contribution: -0.014
